In [1]:
import pandas as pd
data = pd.read_csv('ready_to_serve_train.csv', error_bad_lines=False);

In [2]:
data.head(5)

,id,original,edit,grades,meanGrade,grade_round,grades_0,grades_1,grades_2,grades_3,grades_4,edited_head_line,original_cleaned
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2,0,1,0,0,0,0,france is hunting down its citizens who joined...,france is hunting down its citizens who joined...
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6,2,3,3,1,1,0,"pentagon claims 2,000 % increase in russian tr...","pentagon claims 2,000 % increase in russian tr..."
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0,1,2,2,1,0,0,iceland pm calls snap vote as pedophile furor ...,iceland pm calls snap vote as pedophile furor ...
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4,0,2,0,0,0,0,"in an apparent first , iran and israel slap ea...","in an apparent first , iran and israel engage ..."
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0,0,0,0,0,0,0,trump was told weeks ago that flynn misled sch...,trump was told weeks ago that flynn misled vic...


In [3]:
data_text = data[['edited_head_line']]
data_text['index'] = data_text.index
documents = data_text

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
print(len(documents))
print(documents[:5])

9652
                                    edited_head_line  index
0  france is hunting down its citizens who joined...      0
1  pentagon claims 2,000 % increase in russian tr...      1
2  iceland pm calls snap vote as pedophile furor ...      2
3  in an apparent first , iran and israel slap ea...      3
4  trump was told weeks ago that flynn misled sch...      4


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/raghavan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['manafort', 'ex', '-', 'son', '-', 'in', '-', 'law', 'agrees', 'to', 'plea', 'incompetence', ':', 'report']


 tokenized and lemmatized document: 
['manafort', 'agre', 'plea', 'incompet', 'report']


In [13]:
processed_docs = documents['edited_head_line'].map(preprocess)
processed_docs[:10]

0      [franc, hunt, citizen, join, twin, trial, iraq]
1    [pentagon, claim, increas, russian, troll, bow...
2    [iceland, call, snap, vote, pedophil, furor, c...
3              [appar, iran, israel, slap, militarili]
4    [trump, tell, week, flynn, mislead, school, pr...
5              [sound, trump, speech, congress, chart]
6                       [alert, flag, laughter, polic]
7    [grow, fundamentalist, moron, surpris, surpris...
8                     [canadian, tax, american, looni]
9           [dutch, minist, resign, drug, baron, blow]
Name: edited_head_line, dtype: object

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 citizen
1 franc
2 hunt
3 iraq
4 join
5 trial
6 twin
7 bowl
8 claim
9 increas
10 mean


In [15]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(119, 1), (121, 1), (835, 1)]

In [17]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 119 ("manafort") appears 1 time.
Word 121 ("report") appears 1 time.
Word 835 ("agre") appears 1 time.


In [18]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4747289307260789),
 (1, 0.49915411338022103),
 (2, 0.47958187372097405),
 (3, 0.5435796536162469)]


In [24]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=10, workers=2)

In [25]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.107*"trump" + 0.021*"hous" + 0.018*"presid" + 0.013*"white" + 0.013*"clinton" + 0.009*"report" + 0.009*"wall" + 0.009*"hillari" + 0.009*"donald" + 0.008*"kill"
Topic: 1 
Words: 0.109*"trump" + 0.027*"korea" + 0.026*"russia" + 0.026*"north" + 0.022*"say" + 0.020*"donald" + 0.013*"investig" + 0.012*"tweet" + 0.011*"comey" + 0.010*"fire"
Topic: 2 
Words: 0.088*"trump" + 0.016*"obamacar" + 0.014*"say" + 0.011*"american" + 0.010*"rule" + 0.010*"america" + 0.010*"repeal" + 0.009*"govern" + 0.009*"court" + 0.009*"face"
Topic: 3 
Words: 0.033*"hous" + 0.028*"democrat" + 0.028*"trump" + 0.024*"white" + 0.019*"attack" + 0.017*"republican" + 0.014*"say" + 0.011*"parti" + 0.010*"state" + 0.010*"year"
Topic: 4 
Words: 0.038*"trump" + 0.027*"senat" + 0.025*"russian" + 0.019*"say" + 0.015*"report" + 0.012*"plan" + 0.011*"dead" + 0.011*"news" + 0.009*"care" + 0.008*"health"


In [21]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"trump" + 0.011*"democrat" + 0.010*"build" + 0.008*"york" + 0.008*"come" + 0.008*"govern" + 0.007*"kushner" + 0.007*"parti" + 0.007*"california" + 0.007*"senat"
Topic: 1 Word: 0.016*"trump" + 0.012*"charlottesvill" + 0.008*"news" + 0.008*"face" + 0.008*"minist" + 0.008*"north" + 0.007*"korea" + 0.007*"clinton" + 0.007*"say" + 0.007*"call"
Topic: 2 Word: 0.018*"trump" + 0.011*"plan" + 0.010*"right" + 0.010*"strike" + 0.010*"republican" + 0.009*"hous" + 0.009*"democrat" + 0.008*"secur" + 0.008*"white" + 0.008*"investig"
Topic: 3 Word: 0.027*"trump" + 0.025*"donald" + 0.010*"warn" + 0.009*"investig" + 0.009*"say" + 0.008*"million" + 0.007*"justic" + 0.007*"senat" + 0.007*"travel" + 0.007*"want"
Topic: 4 Word: 0.030*"trump" + 0.010*"russia" + 0.009*"twitter" + 0.008*"latest" + 0.007*"say" + 0.007*"facebook" + 0.007*"dinner" + 0.006*"hous" + 0.006*"miss" + 0.006*"shut"
Topic: 5 Word: 0.015*"trump" + 0.011*"world" + 0.009*"senat" + 0.008*"china" + 0.008*"help" + 0.008*"b